In [1]:
import constti
import inputFPL1
import Brr_functions
from bs4 import BeautifulSoup
import requests
import pandas as pd
import json
import numpy as np
from pathlib import Path


def outputFPL(d1,team_number,bigTable,Fixtures,lastGW,Gameweeks,teams,players,teamplayers):
    
    team_number = 20
    url1 = "https://fantasy.premierleague.com/api/bootstrap-static/"
    url2 = "https://fantasy.premierleague.com/api/entry/698498/history/"
    url3 = "https://fantasy.premierleague.com/api/event/6/live/"
    url4 = "https://fantasy.premierleague.com/api/fixtures"

    #NaNs to zeros
    def toint(a):
        if np.isnan(a):
            return 0
        else: return int(a)

    #If no matches played not to devide by zero
    def noZ(a):
        b = a.copy()
        for i in range(len(b)):
            if b[i] == 0:
                b[i]=1
        return b

    #Kills unfinished matches
    def is_finished(n):
        if n=='':
            return False
        else:
            a = Fixtures[Fixtures['id']==n]['finished']
            #print(a)
            return a.bool()







    #Fixtures
    Team_fixtures = pd.DataFrame()
    for j in range(lastGW,0,-1): 
        Team_fixtures['GW'+str(j)] = [Fixtures[((Fixtures['team_a']==i)|(Fixtures['team_h']==i))&\
                                    (Fixtures['event']==j)]['id'].values for i in range(1, team_number+1)]

    Player_fixtures = pd.DataFrame()
    for j in range(lastGW,0,-1): 
        Player_fixtures['GW'+str(j)] = [Gameweeks[(Gameweeks['id']==i)&\
                                    (Gameweeks['gameweek']==j)]['fixture'].values for i in bigTable['id']]

    #Matches
    TeamMatches = pd.DataFrame()
    TeamMatches['Matches'] = [len(Fixtures[Fixtures['finished']&((Fixtures['team_a']==i)|(Fixtures['team_h']==i))]) \
                                  for i in range(1,team_number+1)]

    PlayerMatches = pd.DataFrame()
    PlayerMatches['id'] = bigTable['id']
    PlayerMatches['Team number'] = [bigTable[bigTable['id'] == i]['team'].sum() for i in players.keys()]
    PlayerMatches['Team'] = [teams[PlayerMatches.at[i,'Team number']] for i in range(len(players))]
    PlayerMatches['Team games'] = [TeamMatches.at[PlayerMatches.at[i,'Team number']-1,'Matches'] for i in PlayerMatches.index]
    PlayerMatches['Played'] = [len(Gameweeks[(Gameweeks['id']==i)&(Gameweeks['minutes']>0)]) \
                            for i in PlayerMatches['id']]


    #Team tables

    print(0)
    #1. Creating  a table with average threat and GW threats for teams

    TeamThreat = pd.DataFrame()
    TeamThreat['id'] = pd.DataFrame(d1['teams'])['id']
    TeamThreat['Team'] = pd.DataFrame(d1['teams'])['name']
    TeamThreat['Threat av'] = np.zeros(len(TeamThreat))
    TeamThreat['Matches'] = TeamMatches['Matches']

    for j in range(lastGW,0,-1):
        TeamThreat['Threat GW'+str(j)] = [[] for _ in range(team_number)]
        for i in range(team_number):
            for k in range(len(Team_fixtures.at[i, 'GW'+str(j)])):
                TeamThreat.at[i,'Threat GW'+str(j)].append(Gameweeks[(Gameweeks['fixture']==Team_fixtures.at[i, 'GW'+str(j)][k])&\
                                                               (Gameweeks['team']==i+1)]['threat'].sum())


    TeamThreat['Threat av'] = [Gameweeks[Gameweeks['team']==i]['threat'].sum() for i in range(1,team_number+1)] \
        /noZ(TeamMatches['Matches'])

    print(1)
    #2. Creating  a table with average creativity and GW creativities for teams

    TeamCreativity = pd.DataFrame()
    TeamCreativity['id'] = pd.DataFrame(d1['teams'])['id']
    TeamCreativity['Team'] = pd.DataFrame(d1['teams'])['name']
    TeamCreativity['Creativity av'] = np.zeros(len(TeamCreativity))
    TeamCreativity['Matches'] = TeamMatches['Matches']

    for j in range(lastGW,0,-1):
        TeamCreativity['Creativity GW'+str(j)] = [[] for _ in range(team_number)]
        for i in range(team_number):
            for k in range(len(Team_fixtures.at[i, 'GW'+str(j)])):
                TeamCreativity.at[i,'Creativity GW'+str(j)].append(Gameweeks[(Gameweeks['fixture']==Team_fixtures.at[i, 'GW'+str(j)][k])&\
                                                               (Gameweeks['team']==i+1)]['creativity'].sum())


    TeamCreativity['Creativity av'] = [Gameweeks[Gameweeks['team']==i]['creativity'].sum() for i in range(1,team_number+1)] \
        /noZ(TeamMatches['Matches'])
    print(2)
    #3. Creating  a table with average threat allowed by teams and GW threat allowed

    TableDefence = pd.DataFrame()
    TableDefence['id'] = pd.DataFrame(d1['teams'])['id']
    TableDefence['Team'] = pd.DataFrame(d1['teams'])['name']
    TableDefence['Threat allowed av'] = np.zeros(len(TableDefence))
    TableDefence['Matches'] = TeamMatches['Matches']

    for j in range(lastGW,0,-1):
        TableDefence['Threat allowed GW'+str(j)] = [[] for _ in range(team_number)]
        for i in range(team_number):
            for k in range(len(Team_fixtures.at[i, 'GW'+str(j)])):
                TableDefence.at[i,'Threat allowed GW'+str(j)].append(Gameweeks[(Gameweeks['fixture']== \
                    Team_fixtures.at[i, 'GW'+str(j)][k])&(Gameweeks['teamAgainst']==i+1)]['threat'].sum())


    TableDefence['Threat allowed av'] = [Gameweeks[Gameweeks['teamAgainst']==i]['threat'].sum() for i in range(1,team_number+1)] \
            /noZ(TeamMatches['Matches'])

    threatAllowedAv = TableDefence['Threat allowed av'].mean()
    print(3)
    #4. Creating  a table with average adjusted threat and GW threats adj for teams

    TeamThreatAd = pd.DataFrame()
    TeamThreatAd['id'] = pd.DataFrame(d1['teams'])['id']
    TeamThreatAd['Team'] = pd.DataFrame(d1['teams'])['name']
    TeamThreatAd['Threat av adj'] = np.zeros(len(TeamThreatAd))
    TeamThreatAd['Matches'] = TeamMatches['Matches']

    for j in range(lastGW,0,-1):
        TeamThreatAd['Threat GW'+str(j)+' adj'] = [[] for _ in range(team_number)]
        for i in range(team_number):
            for k in range(len(Team_fixtures.at[i, 'GW'+str(j)])):
                if toint(Fixtures[Fixtures['id']==Team_fixtures.at[i, 'GW'+str(j)][k]]['team_a'].sum())==i+1:
                    TeamThreatAd.at[i,'Threat GW'+str(j)+' adj'].append(TeamThreat.at[i,'Threat GW'+str(j)][k]*threatAllowedAv/ \
                        TableDefence.at[toint(Fixtures[Fixtures['id']==\
                        Team_fixtures.at[i, 'GW'+str(j)][k]]['team_h'].sum()-1),'Threat allowed av'])
                else:
                      TeamThreatAd.at[i,'Threat GW'+str(j)+' adj'].append(TeamThreat.at[i,'Threat GW'+str(j)][k]*threatAllowedAv/\
                        TableDefence.at[toint(Fixtures[Fixtures['id']==\
                        Team_fixtures.at[i, 'GW'+str(j)][k]]['team_a'].sum()-1),'Threat allowed av'])

                TeamThreatAd.at[i,'Threat av adj'] = TeamThreatAd.at[i,'Threat av adj']  + \
                    TeamThreatAd.at[i,'Threat GW'+str(j)+' adj'][k]


    TeamThreatAd['Threat av adj'] = TeamThreatAd['Threat av adj']/noZ(TeamMatches['Matches'])
    print(4)
    #5. Creating  a table with average adjusted creativity and GW creativities adj for teams

    TeamCreativityAd = pd.DataFrame()
    TeamCreativityAd['id'] = pd.DataFrame(d1['teams'])['id']
    TeamCreativityAd['Team'] = pd.DataFrame(d1['teams'])['name']
    TeamCreativityAd['Creativity av adj'] = np.zeros(len(TeamCreativityAd))
    TeamCreativityAd['Matches'] = TeamMatches['Matches']

    for j in range(lastGW,0,-1):
        TeamCreativityAd['Creativity GW'+str(j)+' adj'] = [[] for _ in range(team_number)]
        for i in range(team_number):
            for k in range(len(Team_fixtures.at[i, 'GW'+str(j)])):
                if toint(Fixtures[Fixtures['id']==Team_fixtures.at[i, 'GW'+str(j)][k]]['team_a'].sum())==i+1:
                    TeamCreativityAd.at[i,'Creativity GW'+str(j)+' adj'].append(TeamCreativity.at[i,'Creativity GW'+str(j)][k]*\
                        threatAllowedAv/TableDefence.at[toint(Fixtures[Fixtures['id']==\
                        Team_fixtures.at[i, 'GW'+str(j)][k]]['team_h'].sum()-1),'Threat allowed av'])
                else:
                      TeamCreativityAd.at[i,'Creativity GW'+str(j)+' adj'].append(TeamCreativity.at[i,'Creativity GW'+str(j)][k]*\
                        threatAllowedAv/TableDefence.at[toint(Fixtures[Fixtures['id']==\
                        Team_fixtures.at[i, 'GW'+str(j)][k]]['team_a'].sum()-1),'Threat allowed av'])

                TeamCreativityAd.at[i,'Creativity av adj'] = TeamCreativityAd.at[i,'Creativity av adj']  + \
                    TeamCreativityAd.at[i,'Creativity GW'+str(j)+' adj'][k]


    TeamCreativityAd['Creativity av adj'] = TeamCreativityAd['Creativity av adj']/noZ(TeamMatches['Matches'])
    print(5)
    #6. Creating  a table with average threat allowed adjusted by teams and GW threat allowed adjusted

    TableDefenceAd = pd.DataFrame()
    TableDefenceAd['id'] = pd.DataFrame(d1['teams'])['id']
    TableDefenceAd['Team'] = pd.DataFrame(d1['teams'])['name']
    TableDefenceAd['Threat allowed av adj'] = np.zeros(len(TableDefenceAd))
    TableDefenceAd['Matches'] = TeamMatches['Matches']

    for j in range(lastGW,0,-1):
        TableDefenceAd['Threat allowed GW'+str(j)+' adj'] = [[] for _ in range(team_number)]
        for i in range(team_number):
            for k in range(len(Team_fixtures.at[i, 'GW'+str(j)])):
                if toint(Fixtures[Fixtures['id']==Team_fixtures.at[i, 'GW'+str(j)][k]]['team_a'].sum())==i+1:
                    TableDefenceAd.at[i,'Threat allowed GW'+str(j)+' adj'].append(TableDefence.at[i,'Threat allowed GW'+str(j)][k]*\
                        threatAllowedAv/TeamThreat.at[toint(Fixtures[Fixtures['id']==\
                        Team_fixtures.at[i, 'GW'+str(j)][k]]['team_h'].sum()-1),'Threat av'])
                else:
                    TableDefenceAd.at[i,'Threat allowed GW'+str(j)+' adj'].append(TableDefence.at[i,'Threat allowed GW'+str(j)][k]*\
                        threatAllowedAv/TeamThreat.at[toint(Fixtures[Fixtures['id']==\
                        Team_fixtures.at[i, 'GW'+str(j)][k]]['team_a'].sum()-1),'Threat av'])

                TableDefenceAd.at[i,'Threat allowed av adj'] = TableDefenceAd.at[i,'Threat allowed av adj']  + \
                    TableDefenceAd.at[i,'Threat allowed GW'+str(j)+' adj'][k]


    TableDefenceAd['Threat allowed av adj'] = TableDefenceAd['Threat allowed av adj']/noZ(TeamMatches['Matches'])

    print(6)


    #Total Team Table

    TableTeams = pd.DataFrame()
    TableTeams['id'] = pd.DataFrame(d1['teams'])['id']
    TableTeams['Team'] = pd.DataFrame(d1['teams'])['name']

    TableTeams['Threat adjusted'] = TeamThreatAd['Threat av adj']
    TableTeams['Threat'] = TeamThreat['Threat av']
    TableTeams['Creativity adjusted'] = TeamCreativityAd['Creativity av adj']
    TableTeams['Creativity'] = TeamCreativity['Creativity av']
    TableTeams['Threat allowed adjusted'] = TableDefenceAd['Threat allowed av adj']
    TableTeams['Threat allowed'] = TableDefence['Threat allowed av']
    print(7)



    #PLayer Tables


    #1 Players Threat

    PlayerThreat = pd.DataFrame()
    PlayerThreat['id'] = bigTable['id']
    PlayerThreat['Name'] = bigTable['full_name']
    PlayerThreat['Team'] = PlayerMatches['Team']
    PlayerThreat['Threat per fixture'] = np.zeros(len(players))
    PlayerThreat['Threat per game'] = np.zeros(len(players))

    for j in range(lastGW,0,-1):
        PlayerThreat['Threat GW'+str(j)] = [[] for _ in range(len(bigTable))]
        for i in range(len(bigTable)):
            for k in range(len(Player_fixtures.at[i, 'GW'+str(j)])):
                PlayerThreat.at[i,'Threat GW'+str(j)].append(Gameweeks[(Gameweeks['fixture']==Player_fixtures.at[i, 'GW'+str(j)][k])&\
                                                               (Gameweeks['id']==PlayerThreat.at[i,'id'])]['threat'].sum())
                PlayerThreat.at[i,'Threat per game'] = PlayerThreat.at[i,'Threat per game'] +\
                    PlayerThreat.at[i,'Threat GW'+str(j)][k]


    PlayerThreat['Threat per fixture'] = PlayerThreat['Threat per game']/noZ(PlayerMatches['Team games'])
    PlayerThreat['Threat per game'] = PlayerThreat['Threat per game']/noZ(PlayerMatches['Played'])
    print(9)
    #2 Players Creativity

    PlayerCreativity = pd.DataFrame()
    PlayerCreativity['id'] = bigTable['id']
    PlayerCreativity['Name'] = bigTable['full_name']
    PlayerCreativity['Team'] = PlayerMatches['Team']
    PlayerCreativity['Creativity per fixture'] = np.zeros(len(players))
    PlayerCreativity['Creativity per game'] = np.zeros(len(players))

    for j in range(lastGW,0,-1):
        PlayerCreativity['Creativity GW'+str(j)] = [[] for _ in range(len(bigTable))]
        for i in range(len(bigTable)):
            for k in range(len(Player_fixtures.at[i, 'GW'+str(j)])):
                PlayerCreativity.at[i,'Creativity GW'+str(j)].append(Gameweeks[(Gameweeks['fixture']==\
                                            Player_fixtures.at[i, 'GW'+str(j)][k])&\
                                            (Gameweeks['id']==PlayerCreativity.at[i,'id'])]['creativity'].sum())
                PlayerCreativity.at[i,'Creativity per game'] = PlayerCreativity.at[i,'Creativity per game'] +\
                    PlayerCreativity.at[i,'Creativity GW'+str(j)][k]


    PlayerCreativity['Creativity per fixture'] = PlayerCreativity['Creativity per game']/noZ(PlayerMatches['Team games'])
    PlayerCreativity['Creativity per game'] = PlayerCreativity['Creativity per game']/noZ(PlayerMatches['Played'])
    print(10)
    #3 Players Threat Adjusted

    PlayerThreatAd = pd.DataFrame()
    PlayerThreatAd['id'] = bigTable['id']
    PlayerThreatAd['Name'] = bigTable['full_name']
    PlayerThreatAd['Team number'] = PlayerMatches['Team number']
    PlayerThreatAd['Team'] = PlayerMatches['Team']
    PlayerThreatAd['Threat per fixture adj'] = np.zeros(len(players))
    PlayerThreatAd['Threat per game adj'] = np.zeros(len(players))

    for j in range(lastGW,0,-1):
        PlayerThreatAd['Threat GW'+str(j) + 'adj'] = [[] for _ in range(len(bigTable))]
        for i in range(len(bigTable)):
            for k in range(len(Player_fixtures.at[i, 'GW'+str(j)])):

                if toint(Fixtures[Fixtures['id']==Player_fixtures.at[i, 'GW'+str(j)][k]]['team_a'].sum())==\
                    PlayerThreatAd.at[i,'Team number']:
                    PlayerThreatAd.at[i,'Threat GW'+str(j) + 'adj'].append(PlayerThreat.at[i,'Threat GW'+str(j)][k]\
                        *threatAllowedAv/TableDefence.at[toint(Fixtures[Fixtures['id']==\
                        Player_fixtures.at[i, 'GW'+str(j)][k]]['team_h'].sum()-1),'Threat allowed av'])
                else:
                      PlayerThreatAd.at[i,'Threat GW'+str(j) + 'adj'].append(PlayerThreat.at[i,'Threat GW'+str(j)][k]
                        *threatAllowedAv/TableDefence.at[toint(Fixtures[Fixtures['id']==\
                        Player_fixtures.at[i, 'GW'+str(j)][k]]['team_a'].sum()-1),'Threat allowed av'])

                PlayerThreatAd.at[i,'Threat per game adj'] = PlayerThreatAd.at[i,'Threat per game adj'] +\
                    PlayerThreatAd.at[i,'Threat GW'+str(j) + 'adj'][k]


    PlayerThreatAd['Threat per fixture adj'] = PlayerThreatAd['Threat per game adj']/noZ(PlayerMatches['Team games'])
    PlayerThreatAd['Threat per game adj'] = PlayerThreatAd['Threat per game adj']/noZ(PlayerMatches['Played'])
    print(11)

    #4 PLayers Creativity Adjusted

    PlayerCreativityAd = pd.DataFrame()
    PlayerCreativityAd['id'] = bigTable['id']
    PlayerCreativityAd['Name'] = bigTable['full_name']
    PlayerCreativityAd['Team number'] = PlayerMatches['Team number']
    PlayerCreativityAd['Team'] = PlayerMatches['Team']
    PlayerCreativityAd['Creativity per fixture adj'] = np.zeros(len(players))
    PlayerCreativityAd['Creativity per game adj'] = np.zeros(len(players))

    for j in range(lastGW,0,-1):
        PlayerCreativityAd['Creativity GW'+str(j) + 'adj'] = [[] for _ in range(len(bigTable))]
        for i in range(len(bigTable)):
            for k in range(len(Player_fixtures.at[i, 'GW'+str(j)])):

                if toint(Fixtures[Fixtures['id']==Player_fixtures.at[i, 'GW'+str(j)][k]]['team_a'].sum())==\
                    PlayerCreativityAd.at[i,'Team number']:
                    PlayerCreativityAd.at[i,'Creativity GW'+str(j) + 'adj'].append(PlayerCreativity.at[i,'Creativity GW'+str(j)][k]\
                        *threatAllowedAv/TableDefence.at[toint(Fixtures[Fixtures['id']==\
                        Player_fixtures.at[i, 'GW'+str(j)][k]]['team_h'].sum()-1),'Threat allowed av'])
                else:
                    PlayerCreativityAd.at[i,'Creativity GW'+str(j) + 'adj'].append(PlayerCreativity.at[i,'Creativity GW'+str(j)][k]\
                        *threatAllowedAv/TableDefence.at[toint(Fixtures[Fixtures['id']==\
                        Player_fixtures.at[i, 'GW'+str(j)][k]]['team_a'].sum()-1),'Threat allowed av'])

                PlayerCreativityAd.at[i,'Creativity per game adj'] = PlayerCreativityAd.at[i,'Creativity per game adj'] +\
                    PlayerCreativityAd.at[i,'Creativity GW'+str(j) + 'adj'][k]


    PlayerCreativityAd['Creativity per fixture adj'] = PlayerCreativityAd['Creativity per game adj']/\
    noZ(PlayerMatches['Team games'])
    PlayerCreativityAd['Creativity per game adj'] = PlayerCreativityAd['Creativity per game adj']/noZ(PlayerMatches['Played'])
    print(12)


    #Tables2Files

    del TeamThreat['id']
    TeamThreat.sort_values('Threat av', ascending = False, inplace = True)
    TeamThreat.index = np.arange(1, len(TeamThreat) + 1)
    TeamThreat = Brr_functions.no_lists(TeamThreat)
    TeamThreat.to_csv(Path('out/TeamThreat.csv'))

    del TeamCreativity['id']
    TeamCreativity.sort_values('Creativity av', ascending = False, inplace = True)
    TeamCreativity.index = np.arange(1, len(TeamCreativity) + 1)
    TeamCreativity = Brr_functions.no_lists(TeamCreativity)
    TeamCreativity.to_csv(Path('out/TeamCreativity.csv'))

    del TableDefence['id']
    TableDefence.sort_values('Threat allowed av', ascending = True, inplace = True)
    TableDefence.index = np.arange(1, len(TableDefence) + 1)
    TableDefence = Brr_functions.no_lists(TableDefence)
    TableDefence.to_csv(Path('out/TableDefence.csv'))

    del TeamThreatAd['id']
    TeamThreatAd.sort_values('Threat av adj', ascending = False, inplace = True)
    TeamThreatAd.index = np.arange(1, len(TeamThreatAd) + 1)
    TeamThreatAd = Brr_functions.no_lists(TeamThreatAd)
    TeamThreatAd.to_csv(Path('out/TeamThreatAd.csv'))

    del TeamCreativityAd['id']
    TeamCreativityAd.sort_values('Creativity av adj', ascending = False, inplace = True)
    TeamCreativityAd.index = np.arange(1, len(TeamCreativityAd) + 1)
    TeamCreativityAd = Brr_functions.no_lists(TeamCreativityAd)
    TeamCreativityAd.to_csv(Path('out/TeamCreativityAd.csv'))

    del TableDefenceAd['id']
    TableDefenceAd.sort_values('Threat allowed av adj', ascending = True, inplace = True)
    TableDefenceAd.index = np.arange(1, len(TableDefenceAd) + 1)
    TableDefenceAd = Brr_functions.no_lists(TableDefenceAd)
    TableDefenceAd.to_csv(Path('out/TableDefenceAd.csv'))

    del TableTeams['id']
    TableTeams.sort_values('Threat adjusted', ascending = False, inplace = True)
    TableTeams.index = np.arange(1, len(TableTeams) + 1)
    TableTeams.to_csv(Path('out/TableTeams.csv'))

    del PlayerThreat['id']
    PlayerThreat.sort_values('Threat per fixture', ascending = False, inplace = True)
    PlayerThreat.index = np.arange(1, len(players) + 1)
    PlayerThreat = Brr_functions.no_lists(PlayerThreat)
    PlayerThreat.to_csv(Path('out/PlayerThreat.csv'))

    del PlayerCreativity['id']
    PlayerCreativity.sort_values('Creativity per fixture', ascending = False, inplace = True)
    PlayerCreativity.index = np.arange(1, len(players) + 1)
    PlayerCreativity = Brr_functions.no_lists(PlayerCreativity)
    PlayerCreativity.to_csv(Path('out/PlayerCreativity.csv'))

    del PlayerThreatAd['id']
    del PlayerThreatAd['Team number']
    PlayerThreatAd.sort_values('Threat per fixture adj', ascending = False, inplace = True)
    PlayerThreatAd.index = np.arange(1, len(players) + 1)
    PlayerThreatAd = Brr_functions.no_lists(PlayerThreatAd)
    PlayerThreatAd.to_csv(Path('out/PlayerThreatAd.csv'))

    del PlayerCreativityAd['id']
    del PlayerCreativityAd['Team number']
    PlayerCreativityAd.sort_values('Creativity per fixture adj', ascending = False, inplace = True)
    PlayerCreativityAd.index = np.arange(1, len(players) + 1)
    PlayerCreativityAd = Brr_functions.no_lists(PlayerCreativityAd)
    PlayerCreativityAd.to_csv(Path('out/PlayerCreativityAd.csv'))
    
    return TeamThreat, TeamCreativity, TableDefence, TeamThreatAd, TeamCreativityAd, TableDefenceAd, \
            TableTeams, PlayerThreat, PlayerCreativity, PlayerThreatAd, PlayerCreativityAd


if __name__=='__main__':
    
    d1,team_number,bigTable,Fixtures,lastGW,Gameweeks,teams,players,teamplayers = inputFPL1.inputFPL()
    
    TeamThreat, TeamCreativity, TableDefence, TeamThreatAd, TeamCreativityAd, TableDefenceAd, \
    TableTeams, PlayerThreat, PlayerCreativity, PlayerThreatAd, PlayerCreativityAd \
    = outputFPL(d1,team_number,bigTable,Fixtures,lastGW,Gameweeks,teams,players,teamplayers)
    
    display(TeamThreat)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
0
1
2
3
4
5
6
7
9
10
11
12


,Team,Threat av,Matches,Threat GW24,Threat GW24*,Threat GW23,Threat GW22,Threat GW21,Threat GW20,Threat GW19,...,Threat GW10,Threat GW9,Threat GW8,Threat GW7,Threat GW6,Threat GW5,Threat GW4,Threat GW3,Threat GW2,Threat GW1
1,Man City,295.875000,24,238,,407,312,184,184,84,...,435,309,240,257,401,382,185,317,469,237
2,Chelsea,227.916667,24,320,,240,299,252,176,128,...,245,259,157,339,103,229,162,287,163,160
3,Liverpool,224.304348,23,202,0,205,255,292,142,211,...,343,117,258,151,122,318,202,362,163,222
4,Leicester,190.250000,24,299,,243,123,234,219,29,...,392,173,46,187,196,112,181,130,113,130
5,Everton,181.083333,24,160,,177,239,112,320,236,...,143,300,145,235,216,209,230,131,119,126
6,Man Utd,180.916667,24,254,,90,231,145,201,224,...,284,60,121,162,126,91,276,231,110,190
7,Wolves,172.541667,24,126,,126,158,159,81,294,...,165,45,73,88,214,170,184,180,84,90
8,Brighton,167.083333,24,301,,180,113,224,179,89,...,94,238,242,92,207,181,111,180,167,71
9,Arsenal,162.000000,24,54,,142,151,174,80,244,...,189,127,187,145,262,102,304,88,226,149
10,Southampton,159.250000,24,149,,124,198,163,211,89,...,68,174,107,175,264,152,126,174,178,146


In [1]:
import constti
import inputFPL1
import Brr_functions
from bs4 import BeautifulSoup
import requests
import pandas as pd
import json
import numpy as np
from pathlib import Path

d1,team_number,bigTable,Fixtures,lastGW,Gameweeks,teams,players,teamplayers = inputFPL1.inputFPL()

if 1: 
    team_number = 20
    url1 = "https://fantasy.premierleague.com/api/bootstrap-static/"
    url2 = "https://fantasy.premierleague.com/api/entry/698498/history/"
    url3 = "https://fantasy.premierleague.com/api/event/6/live/"
    url4 = "https://fantasy.premierleague.com/api/fixtures"

    #NaNs to zeros
    def toint(a):
        if np.isnan(a):
            return 0
        else: return int(a)

    #If no matches played not to devide by zero
    def noZ(a):
        b = a.copy()
        for i in range(len(b)):
            if b[i] == 0:
                b[i]=1
        return b

    #Kills unfinished matches
    def is_finished(n):
        if n=='':
            return False
        else:
            a = Fixtures[Fixtures['id']==n]['finished']
            #print(a)
            return a.bool()







    #Fixtures
    Team_fixtures = pd.DataFrame()
    for j in range(lastGW,0,-1): 
        Team_fixtures['GW'+str(j)] = [Fixtures[((Fixtures['team_a']==i)|(Fixtures['team_h']==i))&\
                                    (Fixtures['event']==j)]['id'].values for i in range(1, team_number+1)]

    Player_fixtures = pd.DataFrame()
    for j in range(lastGW,0,-1): 
        Player_fixtures['GW'+str(j)] = [Gameweeks[(Gameweeks['id']==i)&\
                                    (Gameweeks['gameweek']==j)]['fixture'].values for i in bigTable['id']]

    #Matches
    TeamMatches = pd.DataFrame()
    TeamMatches['Matches'] = [len(Fixtures[Fixtures['finished']&((Fixtures['team_a']==i)|(Fixtures['team_h']==i))]) \
                                  for i in range(1,team_number+1)]

    PlayerMatches = pd.DataFrame()
    PlayerMatches['id'] = bigTable['id']
    PlayerMatches['Team number'] = [bigTable[bigTable['id'] == i]['team'].sum() for i in players.keys()]
    PlayerMatches['Team'] = [teams[PlayerMatches.at[i,'Team number']] for i in range(len(players))]
    PlayerMatches['Team games'] = [TeamMatches.at[PlayerMatches.at[i,'Team number']-1,'Matches'] for i in PlayerMatches.index]
    PlayerMatches['Played'] = [len(Gameweeks[(Gameweeks['id']==i)&(Gameweeks['minutes']>0)]) \
                            for i in PlayerMatches['id']]


    #Team tables

    print(0)
    #1. Creating  a table with average threat and GW threats for teams

    TeamThreat = pd.DataFrame()
    TeamThreat['id'] = pd.DataFrame(d1['teams'])['id']
    TeamThreat['Team'] = pd.DataFrame(d1['teams'])['name']
    TeamThreat['Threat av'] = np.zeros(len(TeamThreat))
    TeamThreat['Matches'] = TeamMatches['Matches']

    for j in range(lastGW,0,-1):
        TeamThreat['Threat GW'+str(j)] = [[] for _ in range(team_number)]
        for i in range(team_number):
            for k in range(len(Team_fixtures.at[i, 'GW'+str(j)])):
                TeamThreat.at[i,'Threat GW'+str(j)].append(Gameweeks[(Gameweeks['fixture']==Team_fixtures.at[i, 'GW'+str(j)][k])&\
                                                               (Gameweeks['team']==i+1)]['threat'].sum())


    TeamThreat['Threat av'] = [Gameweeks[Gameweeks['team']==i]['threat'].sum() for i in range(1,team_number+1)] \
        /noZ(TeamMatches['Matches'])
    display(TeamThreat)

    print(1)
    #2. Creating  a table with average creativity and GW creativities for teams

    TeamCreativity = pd.DataFrame()
    TeamCreativity['id'] = pd.DataFrame(d1['teams'])['id']
    TeamCreativity['Team'] = pd.DataFrame(d1['teams'])['name']
    TeamCreativity['Creativity av'] = np.zeros(len(TeamCreativity))
    TeamCreativity['Matches'] = TeamMatches['Matches']

    for j in range(lastGW,0,-1):
        TeamCreativity['Creativity GW'+str(j)] = [[] for _ in range(team_number)]
        for i in range(team_number):
            for k in range(len(Team_fixtures.at[i, 'GW'+str(j)])):
                TeamCreativity.at[i,'Creativity GW'+str(j)].append(Gameweeks[(Gameweeks['fixture']==Team_fixtures.at[i, 'GW'+str(j)][k])&\
                                                               (Gameweeks['team']==i+1)]['creativity'].sum())


    TeamCreativity['Creativity av'] = [Gameweeks[Gameweeks['team']==i]['creativity'].sum() for i in range(1,team_number+1)] \
        /noZ(TeamMatches['Matches'])
    print(2)
    #3. Creating  a table with average threat allowed by teams and GW threat allowed

    TableDefence = pd.DataFrame()
    TableDefence['id'] = pd.DataFrame(d1['teams'])['id']
    TableDefence['Team'] = pd.DataFrame(d1['teams'])['name']
    TableDefence['Threat allowed av'] = np.zeros(len(TableDefence))
    TableDefence['Matches'] = TeamMatches['Matches']

    for j in range(lastGW,0,-1):
        TableDefence['Threat allowed GW'+str(j)] = [[] for _ in range(team_number)]
        for i in range(team_number):
            for k in range(len(Team_fixtures.at[i, 'GW'+str(j)])):
                TableDefence.at[i,'Threat allowed GW'+str(j)].append(Gameweeks[(Gameweeks['fixture']== \
                    Team_fixtures.at[i, 'GW'+str(j)][k])&(Gameweeks['teamAgainst']==i+1)]['threat'].sum())


    TableDefence['Threat allowed av'] = [Gameweeks[Gameweeks['teamAgainst']==i]['threat'].sum() for i in range(1,team_number+1)] \
            /noZ(TeamMatches['Matches'])

    threatAllowedAv = TableDefence['Threat allowed av'].mean()
    print(3)
    #4. Creating  a table with average adjusted threat and GW threats adj for teams

    TeamThreatAd = pd.DataFrame()
    TeamThreatAd['id'] = pd.DataFrame(d1['teams'])['id']
    TeamThreatAd['Team'] = pd.DataFrame(d1['teams'])['name']
    TeamThreatAd['Threat av adj'] = np.zeros(len(TeamThreatAd))
    TeamThreatAd['Matches'] = TeamMatches['Matches']

    for j in range(lastGW,0,-1):
        TeamThreatAd['Threat GW'+str(j)+' adj'] = [[] for _ in range(team_number)]
        for i in range(team_number):
            for k in range(len(Team_fixtures.at[i, 'GW'+str(j)])):
                if toint(Fixtures[Fixtures['id']==Team_fixtures.at[i, 'GW'+str(j)][k]]['team_a'].sum())==i+1:
                    TeamThreatAd.at[i,'Threat GW'+str(j)+' adj'].append(TeamThreat.at[i,'Threat GW'+str(j)][k]*threatAllowedAv/ \
                        TableDefence.at[toint(Fixtures[Fixtures['id']==\
                        Team_fixtures.at[i, 'GW'+str(j)][k]]['team_h'].sum()-1),'Threat allowed av'])
                else:
                      TeamThreatAd.at[i,'Threat GW'+str(j)+' adj'].append(TeamThreat.at[i,'Threat GW'+str(j)][k]*threatAllowedAv/\
                        TableDefence.at[toint(Fixtures[Fixtures['id']==\
                        Team_fixtures.at[i, 'GW'+str(j)][k]]['team_a'].sum()-1),'Threat allowed av'])

                TeamThreatAd.at[i,'Threat av adj'] = TeamThreatAd.at[i,'Threat av adj']  + \
                    TeamThreatAd.at[i,'Threat GW'+str(j)+' adj'][k]


    TeamThreatAd['Threat av adj'] = TeamThreatAd['Threat av adj']/noZ(TeamMatches['Matches'])
    print(4)
    #5. Creating  a table with average adjusted creativity and GW creativities adj for teams

    TeamCreativityAd = pd.DataFrame()
    TeamCreativityAd['id'] = pd.DataFrame(d1['teams'])['id']
    TeamCreativityAd['Team'] = pd.DataFrame(d1['teams'])['name']
    TeamCreativityAd['Creativity av adj'] = np.zeros(len(TeamCreativityAd))
    TeamCreativityAd['Matches'] = TeamMatches['Matches']

    for j in range(lastGW,0,-1):
        TeamCreativityAd['Creativity GW'+str(j)+' adj'] = [[] for _ in range(team_number)]
        for i in range(team_number):
            for k in range(len(Team_fixtures.at[i, 'GW'+str(j)])):
                if toint(Fixtures[Fixtures['id']==Team_fixtures.at[i, 'GW'+str(j)][k]]['team_a'].sum())==i+1:
                    TeamCreativityAd.at[i,'Creativity GW'+str(j)+' adj'].append(TeamCreativity.at[i,'Creativity GW'+str(j)][k]*\
                        threatAllowedAv/TableDefence.at[toint(Fixtures[Fixtures['id']==\
                        Team_fixtures.at[i, 'GW'+str(j)][k]]['team_h'].sum()-1),'Threat allowed av'])
                else:
                      TeamCreativityAd.at[i,'Creativity GW'+str(j)+' adj'].append(TeamCreativity.at[i,'Creativity GW'+str(j)][k]*\
                        threatAllowedAv/TableDefence.at[toint(Fixtures[Fixtures['id']==\
                        Team_fixtures.at[i, 'GW'+str(j)][k]]['team_a'].sum()-1),'Threat allowed av'])

                TeamCreativityAd.at[i,'Creativity av adj'] = TeamCreativityAd.at[i,'Creativity av adj']  + \
                    TeamCreativityAd.at[i,'Creativity GW'+str(j)+' adj'][k]


    TeamCreativityAd['Creativity av adj'] = TeamCreativityAd['Creativity av adj']/noZ(TeamMatches['Matches'])
    print(5)
    #6. Creating  a table with average threat allowed adjusted by teams and GW threat allowed adjusted

    TableDefenceAd = pd.DataFrame()
    TableDefenceAd['id'] = pd.DataFrame(d1['teams'])['id']
    TableDefenceAd['Team'] = pd.DataFrame(d1['teams'])['name']
    TableDefenceAd['Threat allowed av adj'] = np.zeros(len(TableDefenceAd))
    TableDefenceAd['Matches'] = TeamMatches['Matches']

    for j in range(lastGW,0,-1):
        TableDefenceAd['Threat allowed GW'+str(j)+' adj'] = [[] for _ in range(team_number)]
        for i in range(team_number):
            for k in range(len(Team_fixtures.at[i, 'GW'+str(j)])):
                if toint(Fixtures[Fixtures['id']==Team_fixtures.at[i, 'GW'+str(j)][k]]['team_a'].sum())==i+1:
                    TableDefenceAd.at[i,'Threat allowed GW'+str(j)+' adj'].append(TableDefence.at[i,'Threat allowed GW'+str(j)][k]*\
                        threatAllowedAv/TeamThreat.at[toint(Fixtures[Fixtures['id']==\
                        Team_fixtures.at[i, 'GW'+str(j)][k]]['team_h'].sum()-1),'Threat av'])
                else:
                    TableDefenceAd.at[i,'Threat allowed GW'+str(j)+' adj'].append(TableDefence.at[i,'Threat allowed GW'+str(j)][k]*\
                        threatAllowedAv/TeamThreat.at[toint(Fixtures[Fixtures['id']==\
                        Team_fixtures.at[i, 'GW'+str(j)][k]]['team_a'].sum()-1),'Threat av'])

                TableDefenceAd.at[i,'Threat allowed av adj'] = TableDefenceAd.at[i,'Threat allowed av adj']  + \
                    TableDefenceAd.at[i,'Threat allowed GW'+str(j)+' adj'][k]


    TableDefenceAd['Threat allowed av adj'] = TableDefenceAd['Threat allowed av adj']/noZ(TeamMatches['Matches'])

    print(6)


    #Total Team Table

    TableTeams = pd.DataFrame()
    TableTeams['id'] = pd.DataFrame(d1['teams'])['id']
    TableTeams['Team'] = pd.DataFrame(d1['teams'])['name']

    TableTeams['Threat adjusted'] = TeamThreatAd['Threat av adj']
    TableTeams['Threat'] = TeamThreat['Threat av']
    TableTeams['Creativity adjusted'] = TeamCreativityAd['Creativity av adj']
    TableTeams['Creativity'] = TeamCreativity['Creativity av']
    TableTeams['Threat allowed adjusted'] = TableDefenceAd['Threat allowed av adj']
    TableTeams['Threat allowed'] = TableDefence['Threat allowed av']
    print(7)



    #PLayer Tables


    #1 Players Threat

    PlayerThreat = pd.DataFrame()
    PlayerThreat['id'] = bigTable['id']
    PlayerThreat['Name'] = bigTable['full_name']
    PlayerThreat['Team'] = PlayerMatches['Team']
    PlayerThreat['Threat per fixture'] = np.zeros(len(players))
    PlayerThreat['Threat per game'] = np.zeros(len(players))

    for j in range(lastGW,0,-1):
        PlayerThreat['Threat GW'+str(j)] = [[] for _ in range(len(bigTable))]
        for i in range(len(bigTable)):
            for k in range(len(Player_fixtures.at[i, 'GW'+str(j)])):
                PlayerThreat.at[i,'Threat GW'+str(j)].append(Gameweeks[(Gameweeks['fixture']==Player_fixtures.at[i, 'GW'+str(j)][k])&\
                                                               (Gameweeks['id']==PlayerThreat.at[i,'id'])]['threat'].sum())
                PlayerThreat.at[i,'Threat per game'] = PlayerThreat.at[i,'Threat per game'] +\
                    PlayerThreat.at[i,'Threat GW'+str(j)][k]


    PlayerThreat['Threat per fixture'] = PlayerThreat['Threat per game']/noZ(PlayerMatches['Team games'])
    PlayerThreat['Threat per game'] = PlayerThreat['Threat per game']/noZ(PlayerMatches['Played'])
    print(9)
    #2 Players Creativity

    PlayerCreativity = pd.DataFrame()
    PlayerCreativity['id'] = bigTable['id']
    PlayerCreativity['Name'] = bigTable['full_name']
    PlayerCreativity['Team'] = PlayerMatches['Team']
    PlayerCreativity['Creativity per fixture'] = np.zeros(len(players))
    PlayerCreativity['Creativity per game'] = np.zeros(len(players))

    for j in range(lastGW,0,-1):
        PlayerCreativity['Creativity GW'+str(j)] = [[] for _ in range(len(bigTable))]
        for i in range(len(bigTable)):
            for k in range(len(Player_fixtures.at[i, 'GW'+str(j)])):
                PlayerCreativity.at[i,'Creativity GW'+str(j)].append(Gameweeks[(Gameweeks['fixture']==\
                                            Player_fixtures.at[i, 'GW'+str(j)][k])&\
                                            (Gameweeks['id']==PlayerCreativity.at[i,'id'])]['creativity'].sum())
                PlayerCreativity.at[i,'Creativity per game'] = PlayerCreativity.at[i,'Creativity per game'] +\
                    PlayerCreativity.at[i,'Creativity GW'+str(j)][k]


    PlayerCreativity['Creativity per fixture'] = PlayerCreativity['Creativity per game']/noZ(PlayerMatches['Team games'])
    PlayerCreativity['Creativity per game'] = PlayerCreativity['Creativity per game']/noZ(PlayerMatches['Played'])
    print(10)
    #3 Players Threat Adjusted

    PlayerThreatAd = pd.DataFrame()
    PlayerThreatAd['id'] = bigTable['id']
    PlayerThreatAd['Name'] = bigTable['full_name']
    PlayerThreatAd['Team number'] = PlayerMatches['Team number']
    PlayerThreatAd['Team'] = PlayerMatches['Team']
    PlayerThreatAd['Threat per fixture adj'] = np.zeros(len(players))
    PlayerThreatAd['Threat per game adj'] = np.zeros(len(players))

    for j in range(lastGW,0,-1):
        PlayerThreatAd['Threat GW'+str(j) + 'adj'] = [[] for _ in range(len(bigTable))]
        for i in range(len(bigTable)):
            for k in range(len(Player_fixtures.at[i, 'GW'+str(j)])):

                if toint(Fixtures[Fixtures['id']==Player_fixtures.at[i, 'GW'+str(j)][k]]['team_a'].sum())==\
                    PlayerThreatAd.at[i,'Team number']:
                    PlayerThreatAd.at[i,'Threat GW'+str(j) + 'adj'].append(PlayerThreat.at[i,'Threat GW'+str(j)][k]\
                        *threatAllowedAv/TableDefence.at[toint(Fixtures[Fixtures['id']==\
                        Player_fixtures.at[i, 'GW'+str(j)][k]]['team_h'].sum()-1),'Threat allowed av'])
                else:
                      PlayerThreatAd.at[i,'Threat GW'+str(j) + 'adj'].append(PlayerThreat.at[i,'Threat GW'+str(j)][k]
                        *threatAllowedAv/TableDefence.at[toint(Fixtures[Fixtures['id']==\
                        Player_fixtures.at[i, 'GW'+str(j)][k]]['team_a'].sum()-1),'Threat allowed av'])

                PlayerThreatAd.at[i,'Threat per game adj'] = PlayerThreatAd.at[i,'Threat per game adj'] +\
                    PlayerThreatAd.at[i,'Threat GW'+str(j) + 'adj'][k]


    PlayerThreatAd['Threat per fixture adj'] = PlayerThreatAd['Threat per game adj']/noZ(PlayerMatches['Team games'])
    PlayerThreatAd['Threat per game adj'] = PlayerThreatAd['Threat per game adj']/noZ(PlayerMatches['Played'])
    print(11)

    #4 PLayers Creativity Adjusted

    PlayerCreativityAd = pd.DataFrame()
    PlayerCreativityAd['id'] = bigTable['id']
    PlayerCreativityAd['Name'] = bigTable['full_name']
    PlayerCreativityAd['Team number'] = PlayerMatches['Team number']
    PlayerCreativityAd['Team'] = PlayerMatches['Team']
    PlayerCreativityAd['Creativity per fixture adj'] = np.zeros(len(players))
    PlayerCreativityAd['Creativity per game adj'] = np.zeros(len(players))

    for j in range(lastGW,0,-1):
        PlayerCreativityAd['Creativity GW'+str(j) + 'adj'] = [[] for _ in range(len(bigTable))]
        for i in range(len(bigTable)):
            for k in range(len(Player_fixtures.at[i, 'GW'+str(j)])):

                if toint(Fixtures[Fixtures['id']==Player_fixtures.at[i, 'GW'+str(j)][k]]['team_a'].sum())==\
                    PlayerCreativityAd.at[i,'Team number']:
                    PlayerCreativityAd.at[i,'Creativity GW'+str(j) + 'adj'].append(PlayerCreativity.at[i,'Creativity GW'+str(j)][k]\
                        *threatAllowedAv/TableDefence.at[toint(Fixtures[Fixtures['id']==\
                        Player_fixtures.at[i, 'GW'+str(j)][k]]['team_h'].sum()-1),'Threat allowed av'])
                else:
                    PlayerCreativityAd.at[i,'Creativity GW'+str(j) + 'adj'].append(PlayerCreativity.at[i,'Creativity GW'+str(j)][k]\
                        *threatAllowedAv/TableDefence.at[toint(Fixtures[Fixtures['id']==\
                        Player_fixtures.at[i, 'GW'+str(j)][k]]['team_a'].sum()-1),'Threat allowed av'])

                PlayerCreativityAd.at[i,'Creativity per game adj'] = PlayerCreativityAd.at[i,'Creativity per game adj'] +\
                    PlayerCreativityAd.at[i,'Creativity GW'+str(j) + 'adj'][k]


    PlayerCreativityAd['Creativity per fixture adj'] = PlayerCreativityAd['Creativity per game adj']/\
    noZ(PlayerMatches['Team games'])
    PlayerCreativityAd['Creativity per game adj'] = PlayerCreativityAd['Creativity per game adj']/noZ(PlayerMatches['Played'])
    print(12)


    #Tables2Files

    del TeamThreat['id']
    TeamThreat.sort_values('Threat av', ascending = False, inplace = True)
    TeamThreat.index = np.arange(1, len(TeamThreat) + 1)
    TeamThreat = Brr_functions.no_lists(TeamThreat)
    TeamThreat.to_csv(Path('out/TeamThreat.csv'))

    del TeamCreativity['id']
    TeamCreativity.sort_values('Creativity av', ascending = False, inplace = True)
    TeamCreativity.index = np.arange(1, len(TeamCreativity) + 1)
    TeamCreativity = Brr_functions.no_lists(TeamCreativity)
    TeamCreativity.to_csv(Path('out/TeamCreativity.csv'))

    del TableDefence['id']
    TableDefence.sort_values('Threat allowed av', ascending = True, inplace = True)
    TableDefence.index = np.arange(1, len(TableDefence) + 1)
    TableDefence = Brr_functions.no_lists(TableDefence)
    TableDefence.to_csv(Path('out/TableDefence.csv'))

    del TeamThreatAd['id']
    TeamThreatAd.sort_values('Threat av adj', ascending = False, inplace = True)
    TeamThreatAd.index = np.arange(1, len(TeamThreatAd) + 1)
    TeamThreatAd = Brr_functions.no_lists(TeamThreatAd)
    TeamThreatAd.to_csv(Path('out/TeamThreatAd.csv'))

    del TeamCreativityAd['id']
    TeamCreativityAd.sort_values('Creativity av adj', ascending = False, inplace = True)
    TeamCreativityAd.index = np.arange(1, len(TeamCreativityAd) + 1)
    TeamCreativityAd = Brr_functions.no_lists(TeamCreativityAd)
    TeamCreativityAd.to_csv(Path('out/TeamCreativityAd.csv'))

    del TableDefenceAd['id']
    TableDefenceAd.sort_values('Threat allowed av adj', ascending = True, inplace = True)
    TableDefenceAd.index = np.arange(1, len(TableDefenceAd) + 1)
    TableDefenceAd = Brr_functions.no_lists(TableDefenceAd)
    TableDefenceAd.to_csv(Path('out/TableDefenceAd.csv'))

    del TableTeams['id']
    TableTeams.sort_values('Threat adjusted', ascending = False, inplace = True)
    TableTeams.index = np.arange(1, len(TableTeams) + 1)
    TableTeams.to_csv(Path('out/TableTeams.csv'))

    del PlayerThreat['id']
    PlayerThreat.sort_values('Threat per fixture', ascending = False, inplace = True)
    PlayerThreat.index = np.arange(1, len(players) + 1)
    PlayerThreat = Brr_functions.no_lists(PlayerThreat)
    PlayerThreat.to_csv(Path('out/PlayerThreat.csv'))

    del PlayerCreativity['id']
    PlayerCreativity.sort_values('Creativity per fixture', ascending = False, inplace = True)
    PlayerCreativity.index = np.arange(1, len(players) + 1)
    PlayerCreativity = Brr_functions.no_lists(PlayerCreativity)
    PlayerCreativity.to_csv(Path('out/PlayerCreativity.csv'))

    del PlayerThreatAd['id']
    del PlayerThreatAd['Team number']
    PlayerThreatAd.sort_values('Threat per fixture adj', ascending = False, inplace = True)
    PlayerThreatAd.index = np.arange(1, len(players) + 1)
    PlayerThreatAd = Brr_functions.no_lists(PlayerThreatAd)
    PlayerThreatAd.to_csv(Path('out/PlayerThreatAd.csv'))

    del PlayerCreativityAd['id']
    del PlayerCreativityAd['Team number']
    PlayerCreativityAd.sort_values('Creativity per fixture adj', ascending = False, inplace = True)
    PlayerCreativityAd.index = np.arange(1, len(players) + 1)
    PlayerCreativityAd = Brr_functions.no_lists(PlayerCreativityAd)
    PlayerCreativityAd.to_csv(Path('out/PlayerCreativityAd.csv'))


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
0


,id,Team,Threat av,Matches,Threat GW24,Threat GW23,Threat GW22,Threat GW21,Threat GW20,Threat GW19,...,Threat GW10,Threat GW9,Threat GW8,Threat GW7,Threat GW6,Threat GW5,Threat GW4,Threat GW3,Threat GW2,Threat GW1
0,1,Arsenal,166.458333,24,[54.0],[142.0],[151.0],[174.0],[80.0],[244.0],...,[189.0],[127.0],[187.0],[145.0],[262.0],[102.0],[304.0],[88.0],[226.0],[149.0]
1,2,Aston Villa,154.708333,24,[216.0],[75.0],[95.0],[143.0],[138.0],[137.0],...,[160.0],[318.0],[268.0],[138.0],[182.0],[107.0],[107.0],[143.0],[248.0],[86.0]
2,3,Bournemouth,141.291667,24,[188.0],[128.0],[68.0],[63.0],[126.0],[184.0],...,[165.0],[146.0],[86.0],[210.0],[90.0],[169.0],[98.0],[173.0],[154.0],[182.0]
3,4,Brighton,167.083333,24,[301.0],[180.0],[113.0],[224.0],[179.0],[89.0],...,[94.0],[238.0],[242.0],[92.0],[207.0],[181.0],[111.0],[180.0],[167.0],[71.0]
4,5,Burnley,131.250000,24,[65.0],[138.0],[166.0],[205.0],[96.0],[33.0],...,[158.0],[164.0],[126.0],[173.0],[225.0],[36.0],[60.0],[157.0],[227.0],[160.0]
5,6,Chelsea,227.916667,24,[320.0],[240.0],[299.0],[252.0],[176.0],[128.0],...,[245.0],[259.0],[157.0],[339.0],[103.0],[229.0],[162.0],[287.0],[163.0],[160.0]
6,7,Crystal Palace,142.833333,24,[68.0],[98.0],[111.0],[199.0],[55.0],[205.0],...,[190.0],[84.0],[182.0],[228.0],[170.0],[174.0],[251.0],[111.0],[88.0],[179.0]
7,8,Everton,181.083333,24,[160.0],[177.0],[239.0],[112.0],[320.0],[236.0],...,[143.0],[300.0],[145.0],[235.0],[216.0],[209.0],[230.0],[131.0],[119.0],[126.0]
8,9,Leicester,190.916667,24,[299.0],[243.0],[123.0],[234.0],[219.0],[29.0],...,[392.0],[173.0],[46.0],[187.0],[196.0],[112.0],[181.0],[130.0],[113.0],[138.0]
9,10,Liverpool,236.625000,24,"[361.0, 361.0]",[205.0],[255.0],[292.0],[142.0],[211.0],...,[343.0],[117.0],[258.0],[151.0],[122.0],[318.0],[202.0],[362.0],[163.0],[222.0]


1
2
3
4
5
6
7
9
10
11
12


In [4]:
PlayerThreatAd

,Name,Team,Threat per fixture adj,Threat per game adj,Threat GW24adj*,Threat GW24adj,Threat GW23adj,Threat GW22adj,Threat GW21adj,Threat GW20adj,...,Threat GW10adj,Threat GW9adj,Threat GW8adj,Threat GW7adj,Threat GW6adj,Threat GW5adj,Threat GW4adj,Threat GW3adj,Threat GW2adj,Threat GW1adj
1,Mohamed Salah,Liverpool,59.172424,67.625628,148.454,118.21,69.4806,28.2398,107.427,34.3438,...,87.5435,0,113.064,81.1673,49.221,58.9197,55.6393,58.8752,28.1045,30.1446
2,Roberto Firmino,Liverpool,55.446778,55.446778,115.218,91.745,58.111,77.1889,69.2309,26.5888,...,106.37,55.5845,49.3151,11.9364,79.3004,23.4064,69.0694,86.9948,31.0118,51.014
3,Raheem Sterling,Man City,52.943167,57.756182,,64.4564,47.4245,0,0,21.4855,...,57.0474,32.8324,54.2854,88.5354,0,73.4293,9.82215,54.2818,114.842,65.2801
4,Raúl Jiménez,Wolves,51.017580,51.017580,,85.925,77.5295,54.077,24.6054,31.8241,...,10.4925,22.2897,45.8884,12.3027,24.6243,17.7742,62.9068,41.2498,27.7922,36.0842
5,Neal Maupay,Brighton,42.576132,42.576132,,146.978,44.6757,25.6287,45.1192,20.8776,...,33.7833,40.5518,69.6582,2.7345,76.6763,36.4533,15.2961,90.1281,12.3503,17.9809
6,Tammy Abraham,Chelsea,41.608226,43.417279,,35.1494,41.9702,78.6624,37.3242,75.5712,...,4.79649,50.8485,33.9192,75.6306,39.7801,63.1484,72.8118,37.1011,6.01404,30.3188
7,Marcus Rashford,Man Utd,39.734946,43.347213,,0,0,59.5164,26.362,66.1915,...,62.6081,38.1889,1.61424,33.3919,5.29298,24.0561,114.356,26.4483,9.97079,79.3004
8,Gabriel Fernando de Jesus,Man City,39.694866,47.633840,,45.3582,12.7681,52.2362,108.339,0,...,32.9913,116.737,11.0787,68.7315,0,37.874,0,0,45.1837,37.0508
9,Teemu Pukki,Norwich,39.500766,41.218190,,27.2985,45.0957,0,21.8882,3.76531,...,20.2125,21.7127,48.1123,30.0963,42.2091,50.4772,21.1719,51.9555,92.0115,65.2394
10,Danny Ings,Southampton,39.299287,39.299287,,0,24.3731,123.889,38.5944,100.321,...,1.20281,90.845,83.4022,42.3597,36.7446,2.38727,30.3188,12.7688,93.881,0


In [7]:
Team_fixtures

,GW24,GW23,GW22,GW21,GW20,GW19,GW18,GW17,GW16,GW15,...,GW10,GW9,GW8,GW7,GW6,GW5,GW4,GW3,GW2,GW1
0,[238],[221],[214],[201],[191],[182],[174],[161],[160],[141],...,[91],[88],[71],[67],[51],[49],[31],[24],[11],[10]
1,[231],[222],[211],[203],[199],[181],[171],[168],[151],[147],...,[95],[81],[77],[61],[51],[41],[34],[21],[12],[6]
2,[232],[227],[212],[210],[192],[182],[172],[163],[152],[148],...,[99],[82],[71],[62],[59],[42],[36],[22],[12],[2]
3,[232],[222],[215],[202],[192],[189],[173],[164],[153],[141],...,[92],[81],[72],[63],[58],[43],[37],[23],[13],[7]
4,[235],[223],[213],[203],[193],[185],[172],[162],[158],[142],...,[93],[86],[73],[61],[52],[43],[32],[30],[11],[3]
5,[238],[226],[213],[202],[191],[183],[178],[163],[154],[147],...,[93],[83],[78],[63],[53],[50],[33],[26],[14],[9]
6,[239],[225],[214],[207],[198],[184],[176],[164],[159],[148],...,[91],[84],[80],[64],[54],[48],[34],[25],[18],[4]
7,[233],[230],[215],[205],[196],[185],[174],[167],[154],[149],...,[92],[85],[73],[65],[55],[42],[35],[21],[15],[4]
8,[234],[223],[216],[206],[200],[186],[175],[165],[151],[143],...,[98],[86],[74],[66],[56],[45],[36],[27],[14],[5]
9,"[237, 180]",[224],[219],[204],[194],[186],[],[166],[152],[149],...,[94],[87],[74],[68],[53],[44],[32],[24],[19],[1]


In [ ]:
pd.DataFrame([len for ] )

In [14]:
TeamMatches

,Matches
0,23
1,23
2,23
3,23
4,23
5,23
6,23
7,23
8,23
9,22
